https://www.youtube.com/watch?v=DbESHcCoWbM&t=13s

In [9]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader, Dataset
import pytorch_lightning as pl
from PIL import Image
import os

In [10]:
# import sphere data
# define path
# create train and validation and test sets
# 40, 10, 10

# path on my specific device
train_path = "/Users/maryclaremartin/Documents/Github/Research-Fall-2022/cali_camera_data/train"
validation_path = "/Users/maryclaremartin/Documents/Github/Research-Fall-2022/cali_camera_data/validation"
test_path = "/Users/maryclaremartin/Documents/Github/Research-Fall-2022/cali_camera_data/test"

In [11]:
# create image arrays for use in train, validation, and test dataloaders
# path: path to folder on specific device
def image_arrays(path):
    dir_list = os.listdir(path)
    
    # empty array for image names
    image_names = []
    
    # add image names from directory to a list
    for image in dir_list:
        image_names.append(image)

    # sort image titles
    image_names = sorted(image_names)
    
    #create array for images, segmentation images
    images = []
    seg_images = []

    for image in image_names:
        if "img" in image:
            images.append(Image.open(os.path.join(path,image)))
        elif "seg" in image:
            seg_images.append(Image.open(os.path.join(path,image)))
            
    return images, seg_images

In [12]:
# create arrays for train, validation, test
train_images, seg_train_images = image_arrays(train_path)
validation_images, seg_validation_images = image_arrays(validation_path)
test_images, seg_test_images = image_arrays(test_path)

In [13]:
# define dataset
class sphereDataset(Dataset):
    def __init__(self, images, seg_images, transform=transforms.ToTensor(), target_transform=transforms.ToTensor()):
        self.images = images
        self.seg_images = seg_images
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.seg_images)

    def __getitem__(self, idx):
        image = images[idx]
        seg = seg_images[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            seg = self.target_transform(seg)
        return image, seg
    

In [14]:
# array for storing average epoch loss to plot later
avg_epoch_loss = []
avg_loss = 0

In [15]:
class ImageClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # 1 by 1 convolution
        self.conv = nn.Conv2d(3,1,(1,1))
        
        self.loss = nn.BCEWithLogitsLoss()
    
    def forward(self, x):
        logits = self.conv(x)
        return logits
    
    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=1e-2) 
        return optimizer
    
    def training_step(self, batch, batch_idx):
        x, y = batch

        # forward
        logits = self(x)
        
        # compute the objective function
        J = self.loss(logits, y)
        
        # compute average loss across the epoch
        #if batch_idx == 0:
            #avg_loss = J
        #elif batch_idx == 39:
            #avg_loss = avg_loss / 40
            #avg_epoch_loss.append(avg_loss)
        #else:
            #avg_loss = avg_loss + J
            
        return J
        
    def train_dataloader(self):
        train_data = sphereDataset(train_images, seg_train_images)
        train_loader = DataLoader(train_data, batch_size=1)
        return train_loader
    
    def validation_dataloader(self):
        validation_data = sphereDataset(validation_images, seg_validation_images)
        validation_loader = DataLoader(validation_data, batch_size=1)
        return validation_loader
    
    def test_dataloader(self):
        test_data = sphereDataset(test_images, seg_test_images)
        test_loader = DataLoader(test_data, batch_size=1)
        return test_loader

# initialize model
model = ImageClassifier()
        

In [16]:
trainer = pl.Trainer(max_epochs = 10)
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type              | Params
-------------------------------------------
0 | conv | Conv2d            | 4     
1 | loss | BCEWithLogitsLoss | 0     
-------------------------------------------
4         Trainable params
0         Non-trainable params
4         Total params
0.000     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

NameError: name 'images' is not defined

In [ ]:
print(model.logger)

print out loss for each epoch (PLOT) for train and validation, print out average precision for validation (PLOT)
By Monday
create validation and test set
overall folder - then train, validation, test
run it on test set

overall view of animal tracking
review paper from Dr. Steibel's student
top level view of research problems in computer vision
--organize the types of problems people have done
--example citations from each one (people have done feeder --then citations)
What are the problems people have tried to solve with computer vision? Who are the main players??
NOT DEPTH ONLY PROBLEM
for next friday